import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
# Imports

In [3]:
from haversine import haversine, Unit
import copy
import re

# Making Top NYC Attractions Data
https://www.planetware.com/tourist-attractions-/new-york-city-us-ny-nyc.htm

In [113]:
# using planteware's list for top attractions 
locations = ["Statue of Liberty", "Central Park", "Rockefeller Center & Top of the Rock Observation Deck", 
            "Metropolitan Museum of Art", "Broadway and the Theater District", "Empire State Building",
            "9/11 Memorial and Museum", "High Line", "Times Square", "Brooklyn Bridge",
            "Fifth Avenue", "Grand Central Terminal", "One World Observatory", "The Frick Collection",
            "New York Public Library", "Wall Street", "Radio City Music Hall", "St. Patrick's Cathedral",
            "Carnegie Hall", "Bryant Park"]
# using google's coordinates for each location 
coords = [[40.689249, -74.044479], [40.782629, -73.965290], [40.759137, -73.979495], 
         [40.779437, -73.963244], [40.759003, -73.984487], [40.748481, -73.985654],
         [40.711532, -74.013201], [40.747797, -74.004872], [40.758070, -73.985407], 
         [40.706086, -73.996864], [40.774431, -73.965618], [40.752708, -73.977247], [40.713353, -74.013325],
         [40.771217, -73.967345], [40.752895, -73.981757], [40.706044, -74.008836], [40.760106, -73.979934],
         [40.758571, -73.976195], [40.765101, -73.979941], [40.753751, -73.983662]]

In [114]:
nyc_attractions = pd.DataFrame(index = locations, data=coords, columns = ['Latitude', "Longitude"])

<IPython.core.display.Javascript object>

In [115]:
nyc_attractions.shape

(20, 2)

In [116]:
nyc_attractions.reset_index(inplace=True)

In [117]:
nyc_attractions.head()

,index,Latitude,Longitude
0,Statue of Liberty,40.689249,-74.044479
1,Central Park,40.782629,-73.965290
2,Rockefeller Center & Top of the Rock Observati...,40.759137,-73.979495
3,Metropolitan Museum of Art,40.779437,-73.963244
4,Broadway and the Theater District,40.759003,-73.984487


In [120]:
nyc_attractions.columns = ['Site', 'Latitude', 'Longitude']

# Making Grid of Distances

In [32]:
liberty = [nyc_attractions.iloc[0][0], nyc_attractions.iloc[0][1]]

In [33]:
liberty2 = coords[0]

In [34]:
central = [nyc_attractions.iloc[1][0], nyc_attractions.iloc[1][1]]

In [35]:
haversine(liberty, central, unit=Unit.MILES)

7.669133429189523

In [36]:
liberty == liberty2

True

In [37]:
coords.insert(0, [40.764712, -73.974485])

In [38]:
def get_distance(coords):
    # must return a matrix of integers, will need scaling 
    distance_matrix = []
    # adding depot (the plaza hotel) to first place 
    plaza_hotel = [40.764712, -73.974485]
    if plaza_hotel not in coords:
        coords.insert(0, plaza_hotel)
    for i in range(len(coords)):
        one_row_list = []
        for num in range(i, len(coords)):
            distance = haversine(coords[i], coords[num], unit=Unit.MILES)
            rounded_upscaled_distance = round((distance * 1000)) # scaled to 1000th of a mile
            one_row_list.append(rounded_upscaled_distance)
        distance_matrix.append(one_row_list) # top right triangle of matrix 
     # making each sublist have 20 items
    for i in range(len(distance_matrix)):
        for x in range(i, len(distance_matrix)):
            if distance_matrix[i][x] != 0:
                distance_matrix[x].insert(i, distance_matrix[i][x])
    return distance_matrix

In [39]:
dist_matrix = get_distance(coords)

In [40]:
# test_dist_matrix2 = copy.deepcopy(test_dist_matrix)

In [41]:
# all of these should be shifted to the right, and the number with opposite dimensions should fill in spaces on left
# i.e.: the zeros should be in a diagonal line from top left to bottom right 
# for i in dist_matrix:
#     print(i)

In [42]:
len(dist_matrix)

21

### Make everything to right of zero leftmost element in cascading list

In [43]:
# for i in range(len(test_dist_matrix2)):
#     for x in range(i, len(test_dist_matrix2)):
#         if test_dist_matrix2[i][x] != 0:
#             test_dist_matrix2[x].insert(i, test_dist_matrix2[i][x])

In [44]:
# for row in test_dist_matrix2:
#     print(row)

# Insert into OR-tools (TSP)
original py file: https://github.com/google/or-tools/blob/stable/ortools/constraint_solver/samples/tsp_cities.py

In [45]:
# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# [END import]


# [START data_model]
def create_data_model(input_distance_matrix):
    """Stores the data for the problem."""
    data = {}
#     data['distance_matrix'] = [
#         [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
#         [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
#         [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
#         [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
#         [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
#         [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
#         [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
#         [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
#         [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
#         [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
#         [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
#         [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
#         [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
#     ]  # yapf: disable
    data['distance_matrix'] = input_distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_miles = solution.ObjectiveValue() / 1000
    print('Objective: {} miles'.format(total_miles))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    return total_miles, plan_output
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(dist_matrix)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        miles, route_list = print_solution(manager, routing, solution)
    route_list_text_match = re.match(".+vehicle\s0:\\n\s(.+0)\\n", route_list)
    stops_as_text = route_list_text_match.group(1)
    list_of_stops = stops_as_text.split(" -> ")
    
    return miles, list_of_stops
    # [END print_solution]


if __name__ == '__main__':
    miles, list_of_stops = main()
    
# [END program]

Objective: 17.05 miles
Route for vehicle 0:
 0 -> 18 -> 12 -> 15 -> 20 -> 6 -> 10 -> 16 -> 1 -> 7 -> 13 -> 8 -> 9 -> 5 -> 3 -> 17 -> 19 -> 2 -> 4 -> 11 -> 14 -> 0



# Interpret and Plot results
need to return results from functions, not just print them

##  Function Outputs

In [46]:
miles

17.05

In [47]:
list_of_stops[0:2]

['0', '18']

## Translate Outputs to place names

In [67]:
nyc_attractions.reset_index(inplace=True)

In [176]:
correct_order_of_sites = []
for x in list_of_stops:
    int_x = int(x)
    if int_x >= 1:
        print(nyc_attractions.iloc[int_x-1]['Site'])
        correct_order_of_sites.append(nyc_attractions.iloc[int_x-1]['Site'])

St. Patrick's Cathedral
Grand Central Terminal
New York Public Library
Bryant Park
Empire State Building
Brooklyn Bridge
Wall Street
Statue of Liberty
9/11 Memorial and Museum
One World Observatory
High Line
Times Square
Broadway and the Theater District
Rockefeller Center & Top of the Rock Observation Deck
Radio City Music Hall
Carnegie Hall
Central Park
Metropolitan Museum of Art
Fifth Avenue
The Frick Collection


In [123]:
nyc_attractions.head(2)

,Site,Latitude,Longitude
0,Statue of Liberty,40.689249,-74.044479
1,Central Park,40.782629,-73.965290


## Itinerary

In [108]:
# list_of_stops[:-2]

In [75]:
itinerary_df = pd.DataFrame()
for x in list_of_stops:
    int_x = int(x)
    if int_x >= 1:
        pd.concat([itinerary_df, nyc_attractions.iloc[int_x-1]])
#         print(nyc_attractions.iloc[int_x-1].name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [103]:
nyc_attractions.shape

(20, 3)

In [170]:
itinerary_df = nyc_attractions.copy()

In [171]:
itinerary_df.head(2)

,Site,Latitude,Longitude
0,Statue of Liberty,40.689249,-74.044479
1,Central Park,40.782629,-73.965290


In [172]:
# itinerary_df.iloc[0] = ['Plaza Hotel (start)', 40.764712, -73.974485]
# itinerary_df.iloc[-1] = ['Plaza Hotel (end)', 40.764712, -73.974485]

In [179]:
# add new itin column
itinerary_df['fastest_order'] = [correct_order_of_sites.index(x) + 1 for x in itinerary_df.Site]
# [int(x) for x in list_of_stops if 0 < int(x) ]
# str(list_of_stops

### Adding hotels to itin_df

In [180]:
hotel_dict = {col: None for col in itinerary_df.columns}

In [181]:
first_row = ['Plaza Hotel (start)', 40.764712, -73.974485, 0]

In [182]:
for idx, key in enumerate(hotel_dict.keys()):
    hotel_dict[key] = first_row[idx]

In [183]:
hotel_dict

{'Site': 'Plaza Hotel (start)',
 'Latitude': 40.764712,
 'Longitude': -73.974485,
 'fastest_order': 0}

In [184]:
itinerary_df = itinerary_df.append(hotel_dict, ignore_index=True)

In [185]:
itinerary_df.shape

(21, 4)

In [186]:
hotel_dict2 = hotel_dict

In [187]:
second_row = ['Plaza Hotel (end)', 40.764712, -73.974485, 21]
for idx, key in enumerate(hotel_dict2.keys()):
    hotel_dict2[key] = second_row[idx]

In [188]:
itinerary_df = itinerary_df.append(hotel_dict2, ignore_index=True)

In [165]:
# itinerary_df.set_index('fastest_order', inplace=True)

In [189]:
itinerary_df.sort_values(by='fastest_order')

,Site,Latitude,Longitude,fastest_order
20,Plaza Hotel (start),40.764712,-73.974485,0
17,St. Patrick's Cathedral,40.758571,-73.976195,1
11,Grand Central Terminal,40.752708,-73.977247,2
14,New York Public Library,40.752895,-73.981757,3
19,Bryant Park,40.753751,-73.983662,4
5,Empire State Building,40.748481,-73.985654,5
9,Brooklyn Bridge,40.706086,-73.996864,6
15,Wall Street,40.706044,-74.008836,7
0,Statue of Liberty,40.689249,-74.044479,8
6,9/11 Memorial and Museum,40.711532,-74.013201,9


# Mapping Results

# Version With Capacity Constraints
original py file: https://github.com/google/or-tools/blob/stable/ortools/constraint_solver/samples/vrp_capacity.py